# Anomalies in Time Series Data

**Lesson Goals**

- Use entropy as a quick way to identify fields that may have anomalies. 

- Use statistical properties to flag the data points that deviate from the expected. 

**The Data**

- Logs of API requests to our data containing sales information about our stores and items. 

- Type of target variable: **Continuous** or Discrete

- Type of observations: **Time Series** or Point in Time   


**The Questions**

- Are there unusual IP addresses accessing our data via the API? 

- Have we seen any spikes or unusual patterns in the size of requests? 

- In general: Does this new value deviate from what we would expect based on historical data? If so, is it something to be concerned about? Remember, we aren't detecting anomalies for the sake of detecting anomalies. 


_____________________________


## Wrangle Data

**Prepare Environment**

In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()


**Acquire**

After doing some research, some experimentation of performing actions and watching the logs, we discovered what each of the fields represent. We then parse and name the fields accordingly. 

In [ ]:
colnames=['ip', 'timestamp', 'request_method', 'status', 'size',
          'destination', 'request_agent']
df = pd.read_csv('https://python.zach.lol/access.log',          
                 engine='python',
                 header=None,
                 index_col=False,
                 names=colnames,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                 na_values='"-"',
                 usecols=[0, 3, 4, 5, 6, 7, 8]
)

For this research, we are only interested in the IP address, timestamp and size of the requests. 

In [ ]:
df = df[['ip', 'timestamp', 'size']]

**Explore IP Address**

In this sample data, it's pretty easy to take a look at value counts to see those IP's that are rare. However, usually the data is much, much larger and looking at simple value counts is not going to be enough. 

In [ ]:
# value counts

In [ ]:
def compute_entropy(series):
    counts = series.value_counts()
    if len(counts)==1:
        ent = 0
    else:
        value, counts = np.unique(series, return_counts=True)
        ent = entropy(counts, base=None)
    return ent

In [ ]:
# len(df.ip.value_counts())

# value, counts = np.unique(df.ip, return_counts=True)
# ent = entropy(counts, base=None)

In [ ]:
compute_entropy(df.ip)

**Prepare Data to Explore Size**

First, we will resample the existing data to 30 minute increments. 

In [ ]:
df.head()

In [ ]:
# select timestamp
df.timestamp = df.timestamp.str.replace(r'(\[|\])', '', regex=True)
df.timestamp = pd.to_datetime(df.timestamp.str.replace(':', ' ', 1))
df = df.set_index('timestamp')
df = df[['size']].resample('1d').sum()

In [ ]:
df = df.fillna(value=0)
df.describe()

**Aside: Simulate some new data to manufacture some anomalies**

Now, let's create a new dataframe that extends our data another year or so. 

In [ ]:
new = pd.DataFrame([["[18/Apr/2019:00:00:00+0000]", 0],
                    ["[15/Mar/2020:00:00:00+0000]", 0]], columns=['timestamp','size'])

We will then resample 

In [ ]:
new.timestamp = new.timestamp.str.replace(r'(\[|\])', '', regex=True)
new.timestamp = pd.to_datetime(new.timestamp.str.replace(':', ' ', 1))
new = new.set_index('timestamp')
new = new.resample('1d').sum()

In [ ]:
new = new.fillna(value=0)
new.tail()

In [ ]:
# get mean and standard deviation for randomly generating some data. 
mean = df['size'].mean()
std = df['size'].std()

Fill values with random number between `[0, mean+2*standard deviation]`. 

In [ ]:
# new['size'] = new['size'].apply(lambda x: max(np.rint(np.random.normal(mean, std)), 0) if x==0 else x)
new['size'] = new['size'].apply(lambda x: np.random.randint(0, mean+2*std) if x==0 else x)

In [ ]:
new.describe()

Fill with some anomalies by replacing the zeros that remain with random number between `[(mean+2*std), (mean+5*std)]`

In [ ]:
new['size'] = new['size'].apply(lambda x: np.random.randint(mean+5*std, mean+9*std) if x<200000000 else x)

In [ ]:
new.describe()

Concatentate our new data with our original data

In [ ]:
df = pd.concat([df, new])

In [ ]:
plt.hist(df['size'])

## Explore Size

First, let's represent size in MB for ease of conceptual understanding.  

In [ ]:
df['size_mb'] = [n/1024/1024 for n in df['size']]
df = df[['size_mb']]

In [ ]:
df.describe()

**Split into Train/Test**

In [ ]:
train = df[:'2019-10-17']
validate = df['2019-10-18':'2020-01-15']
test = df['2020-01-16':]

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(train)
plt.plot(validate)
plt.plot(test)
plt.show()

## Simple Moving Average

In [ ]:
# 1 week
sma_short = train.rolling(window=7).mean()
sma_short[:10]

In [ ]:
sma_long = train.rolling(window=30).mean()
sma_long[27:33]

**Plot the SMA**

In [ ]:

fig, ax = plt.subplots(figsize=(12,4))

ax.plot(train.index, train, label='Size (MB)')

ax.plot(train.index, sma_short, label = '7-day SMA')

ax.plot(train.index, sma_long, label = '30-day SMA')

ax.legend(loc='best')
ax.set_ylabel('Size (MB)')
# ax.xaxis.(rotate=90)
# ax.xaxis.set_major_formatter(my_datetime_fmt)

Try some other windows to compare. 

## Exponential Moving Average

SMA time series are much less noisy than the time series of the original data points. 
The challenge with SMA, however, is that the values of SMA lag the original values. This means that changes in the trend are only seen with a delay (lag) of L time units. 

Exponential Moving Average (EMA) helps reduce the lag induced by the use of the SMA. It does this by putting more weight on more recent observations, while the SMA weights all observations equally.

The EMA function looks like this: 

$EMA_{t}= \alpha * (t_{0} - EMA_{t-1}) + EMA_{t-1}$

Where: 

- M = Number of time periods, span of the window

- $t_{0}$ = Latest value

- $t-1$ = Previous value

- $EMA_{t-1}$ = Exponential moving average of previous day. 

- The multiplier: $\alpha = \frac{2}{M+1}$

However, we will use the pandas ewm (Exponential Weighted functions) to compute our EMA. 
So we just need to define the following: 

- M = `span` argument = number of time periods. We will use 1 week, which is $24*2*7 = 336$

In [ ]:
ema_short = train.ewm(span=7).mean()
ema_short.tail()

In [ ]:
ema_long = train.ewm(span=30).mean()
ema_long.tail()

**Comparison of SMA and EMA**

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(train.index, train, label='Size (MB)', alpha=.5)

ax.plot(train.index, sma_short, label = '7-day SMA')
ax.plot(train.index, ema_short, label = '7-day EMA')

ax.legend(loc='best')
ax.set_ylabel('Size (MB)')

plt.show()

fig, ax = plt.subplots(figsize=(12,6))

ax.plot(train.index, train, label='Size (MB)', alpha=.5)

ax.plot(train.index, sma_long, label = '30-day SMA')
ax.plot(train.index, ema_long, label = '30-day EMA')

ax.legend(loc='best')
ax.set_ylabel('Size (MB)')

plt.show()

# ax.xaxis.(rotate=90)
# ax.xaxis.set_major_formatter(my_datetime_fmt)


## Bollinger Bands and %b

**Bollinger Bands**

- a volatility indicator and commonly used in stock market trading. 

- Made up of 3 lines, the Upper Band (UB), the Lower Band (LB) and the Midband.  

**Midband**

- The Exponential Moving Average

- `midband = train.ewm(span=336).mean()`

**Upper & Lower Band**

- UB/LB = Midband +/- stdev * K

- `stdev = train.ewm(span=336).std()` 

- K = the number of standard deviations to go up and down from the EMA

**%b, Percent Bandwidth**

- Shows where the last value sits in relation to the bands

- $\%b = \frac{last-LB}{UB-LB}$ 

- %b > 1 => point lies above UB

- %b < 0 => point lies below LB

- %b == .5 => point lies on the midband. 

**Bandwidth** 

- The width of the bands

- $Bandwidth = \frac{(UB-LB)}{Midband}$


In [ ]:
# set the window span
span = 30

# compute midband
midband = train.ewm(span=span).mean()

# compute exponential stdev
stdev = train.ewm(span=span).std()

# compute upper and lower bands
ub = midband + stdev*3
lb = midband - stdev*3

Plot the bands

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(train.index, 
        train,
        label='Size (MB)')

ax.plot(train.index, 
        midband, 
        label = '1-week EMA/midband')
ax.plot(train.index, 
        ub, 
        label = 'Upper Band')
ax.plot(train.index, 
        lb, 
        label = 'Lower Band')

ax.legend(loc='best')
ax.set_ylabel('Size (MB)')

Where do you think we will have a %b > 1? 

Compute %b

$\%b = \frac{last-LB}{UB-LB}$ 

In [ ]:
train = pd.concat([train, midband, ub, lb], axis=1)
train.columns = ['size_mb', 'midband', 'ub', 'lb']

In [ ]:
train['pct_b'] = (train['size_mb'] - train['lb'])/(train['ub'] - train['lb'])

In [ ]:
train[train['pct_b']>.9]

## Exercises

**file name:** time_series_anomaly_detection.py or time_series_anomaly_detection.ipynb

Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? Use time series method for detecting anomalies, like exponential moving average with %b.

Bonus:

Can you label students who are viewing both the web dev and data science curriculum?
Can you label students by the program they are in? 
Can you label users by student vs. staff?
What are Zach, Maggie, Faith, and Ryan's ids?

We are now going to fabricate some anomalous observations for demonstration purposes as we work through the lesson. 